In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

    

import torchvision
import torchvision.transforms as transforms

from models import *

In [14]:
batch_size = 128
normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])

train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
print_freq = 100

def train(trainloader, model, criterion, optimizer, epoch, enablePrint=False):
    batch_time = AverageMeter()   ## at the begining of each epoch, this should be reset
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()  # measure current time
    
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)  # data loading time

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end) # time spent to process one batch
        end = time.time()

        if enablePrint and i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

In [8]:
def validate(val_loader, model, criterion,enablePrint=False):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if enablePrint and i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    if enablePrint: print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg

In [9]:
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True) # topk(k, dim=None, largest=True, sorted=True)
                                               # will output (max value, its index)
    pred = pred.t()           # transpose
    correct = pred.eq(target.view(1, -1).expand_as(pred))   # "-1": calculate automatically

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)  # view(-1): make a flattened 1D tensor
        res.append(correct_k.mul_(100.0 / batch_size))   # correct: size of [maxk, batch_size]
    return res

In [10]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n    ## n is impact factor
        self.count += n
        self.avg = self.sum / self.count

In [11]:
def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [80, 120]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1  

In [12]:
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))

In [4]:
model_name = "vggnet16"
model = VGG16()

In [5]:
lr = 0.1
weight_decay = 1e-4
epochs = 150
best_prec = 0

model = model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
# weight decay: for regularization to prevent overfitting


if not os.path.exists('result'):
    os.makedirs('result')
    
fdir = 'result/'+str(model_name)

if not os.path.exists(fdir):
    os.makedirs(fdir)

In [34]:
for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch, enablePrint=False)
    
    # evaluate on test set
    # print("Validation starts")
    prec = validate(testloader, model, criterion, enablePrint=False)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    if epoch % 5 == 0:
        print('Epoch: {epoch}, best acc: {best_prec:.3f}'.format(epoch=epoch, best_prec=best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

Epoch: 0, best acc: 37.630
Epoch: 5, best acc: 70.310
Epoch: 10, best acc: 80.070
Epoch: 15, best acc: 83.060
Epoch: 20, best acc: 85.900
Epoch: 25, best acc: 85.900
Epoch: 30, best acc: 86.900
Epoch: 35, best acc: 87.350
Epoch: 40, best acc: 88.140
Epoch: 45, best acc: 88.350
Epoch: 55, best acc: 88.350
Epoch: 60, best acc: 89.660
Epoch: 65, best acc: 89.660
Epoch: 70, best acc: 89.660
Epoch: 75, best acc: 89.660
Epoch: 80, best acc: 92.540
Epoch: 85, best acc: 92.840
Epoch: 90, best acc: 92.900
Epoch: 95, best acc: 93.140
Epoch: 100, best acc: 93.140
Epoch: 105, best acc: 93.140
Epoch: 110, best acc: 93.140
Epoch: 115, best acc: 93.140
Epoch: 120, best acc: 93.140
Epoch: 125, best acc: 93.140
Epoch: 130, best acc: 93.140
Epoch: 135, best acc: 93.150
Epoch: 140, best acc: 93.160
Epoch: 145, best acc: 93.160


In [15]:
fileStr = f'{fdir}/model_best.pth.tar'
importedModel = VGG16()
importedModel.cuda()
checkpoint = torch.load(fileStr)
importedModel.load_state_dict(checkpoint['state_dict'])
validate(testloader, importedModel, criterion)

93.16

In [16]:
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()

for layer in importedModel.modules():
    if isinstance(layer, torch.nn.Conv2d):
        layer.register_forward_pre_hook(save_output)       ## Input for the module will be grapped       
####################################################

dataiter = iter(trainloader)
images, labels = next(dataiter)
images = images.cuda()
out = importedModel(images)
print("1st convolution's input size:", save_output.outputs[0][0].size())
print("2nd convolution's input size:", save_output.outputs[1][0].size())

1st convolution's input size: torch.Size([128, 3, 32, 32])
2nd convolution's input size: torch.Size([128, 64, 32, 32])


In [46]:
conv1 = importedModel.features[0]
bn1 = importedModel.features[1]

In [48]:
my_input = save_output.outputs[0][0]
my_output = F.relu(F.batch_norm(conv1(my_input), bn1.running_mean, bn1.running_var, weight=bn1.weight, bias=bn1.bias))

In [49]:
(my_output - save_output.outputs[1][0]).sum()

tensor(0., device='cuda:0', grad_fn=<SumBackward0>)